# Regex-Analyse: Produktbestellungen (csv/orders.csv)

Dieses Notebook liest `./csv/orders.csv` als Text und extrahiert Felder per **regulaeren Ausdruecken**.

Hinweis: Die Regex-Muster sind bewusst robust gehalten. Falls das CSV ein anderes Schema hat, passe die Muster im Abschnitt *Konfiguration* an.

In [ ]:
import re
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path

ORDERS_PATH = Path('./csv/orders.csv')

text = ORDERS_PATH.read_text(encoding='utf-8')
print(f'Read {len(text)} characters from {ORDERS_PATH}')

## Konfiguration (Regex-Muster)

Die folgenden Muster decken gaengige Formate ab:

- **Bestellnummern**: reine Ziffernfolgen oder `ORD-1234`/`ORDER-1234`
- **Produktcodes**: Grossbuchstaben + Ziffern, optional mit Bindestrich (z.B. `ABC-123`, `PRD001`)
- **Preise**: `$123.45` oder `123.45`
- **Datum**: `YYYY-MM-DD` oder `MM/DD/YYYY`

Wenn dein Datensatz klar definierte Spalten hat, ist ein *Zeilen-Regex* (siehe unten) am genauesten.

In [ ]:
# Feld-Regex (globale Extraktion aus dem Text)
ORDER_NO_RE   = re.compile(r'\b(?:ORD(?:ER)?[- ]?)?\d{4,}\b')
PRODUCT_RE    = re.compile(r'\b[A-Z]{2,10}(?:-?\d{2,6})\b')
PRICE_RE      = re.compile(r'(?:\$\s*)?(\d+(?:\.\d{2})?)')
DATE_RE       = re.compile(r'\b(?:\d{4}-\d{2}-\d{2}|\d{2}/\d{2}/\d{4})\b')

order_numbers = ORDER_NO_RE.findall(text)
product_codes = PRODUCT_RE.findall(text)
prices_raw    = PRICE_RE.findall(text)
dates         = DATE_RE.findall(text)

print('Order numbers (sample):', order_numbers[:10])
print('Product codes  (sample):', product_codes[:10])
print('Prices        (sample):', prices_raw[:10])
print('Dates         (sample):', dates[:10])

## 1) Alle Bestellnummern extrahieren

In [ ]:
unique_orders = sorted(set(order_numbers))
unique_orders

## 2) Alle Produktcodes extrahieren

In [ ]:
unique_products = sorted(set(product_codes))
unique_products

## 3) Alle Preise extrahieren

In [ ]:
# Konvertiere in float (enthaelt ggf. auch andere Zahlen im CSV; besser ist die Zeilen-Regex unten)
prices = [float(p) for p in prices_raw]
prices[:20], len(prices)

## 4) Alle Bestelldaten extrahieren

In [ ]:
sorted(set(dates))

## Zeilenweises Parsing (empfohlen)

Wenn das CSV pro Zeile genau eine Bestellung enthaelt, ist ein Zeilen-Regex am saubersten.

Unten wird versucht, pro Zeile folgende Felder zu finden:

- `order_no` (Bestellnummer)
- `product_code`
- `price` (float)
- `date` (string)
- `qty` (Anzahl)

Passe das Pattern ggf. an die Spaltenreihenfolge deines CSV an.

In [ ]:
@dataclass(frozen=True)
class Order:
    order_no: str
    product_code: str
    price: float
    date_raw: str
    qty: int

# Beispiel-Pattern: order_no,product_code,price,date,qty  (Kommas als Trenner)
LINE_RE = re.compile(
    r'^\s*'
    r'(?P<order_no>(?:ORD(?:ER)?[- ]?)?\d{4,})\s*,\s*'
    r'(?P<product_code>[A-Z]{2,10}(?:-?\d{2,6}))\s*,\s*'
    r'\$?\s*(?P<price>\d+(?:\.\d{2})?)\s*,\s*'
    r'(?P<date>(?:\d{4}-\d{2}-\d{2}|\d{2}/\d{2}/\d{4}))\s*,\s*'
    r'(?P<qty>\d+)\s*'
    r'$'
)

orders = []
for i, line in enumerate(text.splitlines(), start=1):
    m = LINE_RE.match(line)
    if not m:
        continue
    orders.append(Order(
        order_no=m.group('order_no'),
        product_code=m.group('product_code'),
        price=float(m.group('price')),
        date_raw=m.group('date'),
        qty=int(m.group('qty')),
    ))

len(orders), orders[:5]

## 5) Bestellungen mit Preis ueber 500 $

In [ ]:
expensive = [o for o in orders if o.price > 500]
expensive

## 6) Datumsformat in TT/MM/JJJJ aendern (re.sub)

Wir unterstuetzen zwei Eingabeformate:

- `YYYY-MM-DD` -> `DD/MM/YYYY`
- `MM/DD/YYYY` -> `DD/MM/YYYY`

In [ ]:
def to_ddmmyyyy_in_text(s: str) -> str:
    # YYYY-MM-DD -> DD/MM/YYYY
    s = re.sub(r'\b(\d{4})-(\d{2})-(\d{2})\b', r'\3/\2/\1', s)
    # MM/DD/YYYY -> DD/MM/YYYY
    s = re.sub(r'\b(\d{2})/(\d{2})/(\d{4})\b', r'\2/\1/\3', s)
    return s

converted_text = to_ddmmyyyy_in_text(text)
# Zeige eine kleine Stichprobe: erste 20 Zeilen
print('--- BEFORE ---')
print('\n'.join(text.splitlines()[:20]))
print('--- AFTER ---')
print('\n'.join(converted_text.splitlines()[:20]))

## 7) Bestellungen mit der hoechsten Anzahl bestellter Artikel

In [ ]:
if orders:
    max_qty = max(o.qty for o in orders)
    max_qty_orders = [o for o in orders if o.qty == max_qty]
    max_qty, max_qty_orders
else:
    'Keine passenden Zeilen mit LINE_RE gefunden. Passe das Pattern an.'

## 8) Guenstigste Bestellung(en) (min)

In [ ]:
if orders:
    min_price = min(o.price for o in orders)
    cheapest = [o for o in orders if o.price == min_price]
    min_price, cheapest
else:
    'Keine passenden Zeilen mit LINE_RE gefunden. Passe das Pattern an.'